In [8]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pathlib

from git.util import T
import pytest
import xarray as xr

import mesmer
import mesmer.mesmer_x

# TODO: extend to more scenarios and members
# TODO: extend predictors

In [10]:
scenario = "ssp585"
target_name = "tasmax"
option_2ndfit = False
update_expected_files = False

In [11]:
# set some configuration parameters
THRESHOLD_LAND = 1 / 3
esm = "IPSL-CM6A-LR"

# TODO: replace with filefinder later
# load data
TEST_DATA_PATH = pathlib.Path("/Users/vbauer/Developer/mesmer/tests/test-data")
TEST_PATH = (
    TEST_DATA_PATH / "output" / target_name / "one_scen_one_ens" / "test-params"
)
cmip6_data_path = mesmer.example_data.cmip6_ng_path()


In [12]:
# load predictor data
path_tas = cmip6_data_path / "tas" / "ann" / "g025"

fN_hist = path_tas / f"tas_ann_{esm}_historical_r1i1p1f1_g025.nc"
fN_ssp585 = path_tas / f"tas_ann_{esm}_{scenario}_r1i1p1f1_g025.nc"

time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
tas_hist = xr.open_dataset(fN_hist, decode_times=time_coder).drop_vars(
    ["height", "file_qf", "time_bnds"]
)
tas_ssp585 = xr.open_dataset(fN_ssp585, decode_times=time_coder).drop_vars(
    ["height", "file_qf", "time_bnds"]
)

tas = xr.DataTree.from_dict(
    {
        "historical": tas_hist,
        "ssp585": tas_ssp585,
    }
)

In [13]:
# make global mean
# global_mean_dt = map_over_subtree(mesmer.weighted.global_mean)
tas_glob_mean = mesmer.weighted.global_mean(tas)

# load target data
path_target = cmip6_data_path / target_name / "ann" / "g025"

fN_hist = path_target / f"{target_name}_ann_{esm}_historical_r1i1p1f1_g025.nc"
fN_ssp585 = path_target / f"{target_name}_ann_{esm}_{scenario}_r1i1p1f1_g025.nc"

time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
targ_hist = xr.open_dataset(fN_hist, decode_times=time_coder)
targ_ssp585 = xr.open_dataset(fN_ssp585, decode_times=time_coder)
# make sure times align
targ_hist["time"] = tas_hist["time"]
targ_ssp585["time"] = tas_ssp585["time"]

target_data = xr.DataTree.from_dict(
    {
        "historical": targ_hist,
        "ssp585": targ_ssp585,
    }
)

In [14]:
# stack target data
def mask_and_stack(ds, threshold_land):
    ds = mesmer.mask.mask_ocean_fraction(ds, threshold_land)
    ds = mesmer.mask.mask_antarctica(ds)
    ds = mesmer.grid.stack_lat_lon(ds, stack_dim="gridpoint")
    return ds

# mask_and_stack_dt = map_over_subtree(mask_and_stack)
target_data = mask_and_stack(target_data, threshold_land=THRESHOLD_LAND)
pred_data = xr.DataTree.from_dict({"tas": tas_glob_mean})

# stack datasets
# weights
weights = mesmer.mesmer_x.get_weights_density(
    pred_data=pred_data,
    predictor="tas",
    targ_data=target_data,
    target=target_name,
    dims=("member", "time"),
)

# stacking
stacked_pred, stacked_targ, stacked_weights = (
    mesmer.core.datatree.broadcast_and_stack_scenarios(
        predictors=pred_data,
        target=target_data,
        weights=weights,
        member_dim=None,
        )
)

In [15]:
# declaring analytical form of the conditional distribution
expr = "norm(loc=c1 + c2 * __tas__, scale=c3)"
expr_name = "expr1"
expression_fit = mesmer.mesmer_x.Expression(expr, expr_name, boundaries_params={}, boundaries_coeffs={})
expression_fit.boundaries_params

{'loc': [-inf, inf], 'scale': [0, inf]}

In [ ]:
# preparing optimizers that will be used for first guess and training
myoptions = mesmer.mesmer_x.ConditionalDistributionOptions(
    expression_fit,
    options_optim=None,
    options_solver=None,
)

distrib = mesmer.mesmer_x.ConditionalDistribution(
    expression=expression_fit, options = myoptions
)

In [18]:
# find first guess
coeffs_fg = mesmer.mesmer_x.find_first_guess(
    conditional_distrib = distrib,
    predictors=stacked_pred,
    target=stacked_targ.tasmax,
    weights=stacked_weights.weight,
)
coeffs_fg

<xarray.Dataset> Size: 5kB
Dimensions:  (gridpoint: 118)
Coordinates:
    lat      (gridpoint) float64 944B -49.5 -40.5 -31.5 -31.5 ... 76.5 76.5 76.5
    lon      (gridpoint) float64 944B 288.0 288.0 18.0 ... 306.0 324.0 342.0
Dimensions without coordinates: gridpoint
Data variables:
    c1       (gridpoint) float64 944B 8.99 -14.46 81.14 ... 143.7 84.67 -357.1
    c2       (gridpoint) float64 944B 0.9826 1.094 0.7826 ... 0.6549 2.213
    c3       (gridpoint) float64 944B 1.408 1.2 0.727 ... 0.6723 1.002 1.725

In [19]:
# training the conditional distribution
# first round
distrib.fit(
    predictors=stacked_pred,
    target=stacked_targ.tasmax,
    weights=stacked_weights.weight,
    first_guess=coeffs_fg,
)
transform_coeffs = distrib.coefficients

In [20]:
# second round if necessary
if option_2ndfit:
    transform_coeffs = distrib.fit(
        predictors=stacked_pred,
        target=stacked_targ.tasmax,
        first_guess=transform_coeffs,
        weights=stacked_weights.weight,
        sample_dim="sample",
        option_smooth_coeffs=True,
        r_gasparicohn=500,
    )

transform_coeffs

<xarray.Dataset> Size: 5kB
Dimensions:  (gridpoint: 118)
Coordinates:
    lat      (gridpoint) float64 944B -49.5 -40.5 -31.5 -31.5 ... 76.5 76.5 76.5
    lon      (gridpoint) float64 944B 288.0 288.0 18.0 ... 306.0 324.0 342.0
Dimensions without coordinates: gridpoint
Data variables:
    c1       (gridpoint) float64 944B 8.99 -14.46 81.14 ... 143.7 84.67 -357.1
    c2       (gridpoint) float64 944B 0.9826 1.094 0.7826 ... 0.6549 2.213
    c3       (gridpoint) float64 944B 1.408 1.2 0.727 ... 0.6723 1.002 1.725
Attributes:
    expression_name:  expr1
    expression:       norm(loc=c1 + c2 * __tas__, scale=c3)

In [21]:
# probability integral transform on non-stacked data for AR(1) process
from mesmer.mesmer_x import ConditionalDistribution, ConditionalDistributionOptions, Expression


target_expression = Expression("norm(loc=0, scale=1)", "normal_dist")

pit = mesmer.mesmer_x.ProbabilityIntegralTransform(
    distrib_orig=distrib,
    distrib_targ=ConditionalDistribution(target_expression, ConditionalDistributionOptions(target_expression)),
)
transf_target = pit.transform(
    data=target_data, target_name=target_name, preds_orig=pred_data, preds_targ=None
)

In [24]:
# training of auto-regression with spatially correlated innovations
local_ar_params = mesmer.stats.fit_auto_regression_scen_ens(
    transf_target,
    ens_dim="member",
    dim="time",
    lags=1,
)

# estimate covariance matrix
# prep distance matrix
geodist = mesmer.core.geospatial.geodist_exact(
    lon=target_data["historical"].lon, lat=target_data["historical"].lat
)
# prep localizer
LOCALISATION_RADII = range(1750, 2001, 250)
phi_gc_localizer = mesmer.stats.gaspari_cohn_correlation_matrices(
    geodist=geodist, localisation_radii=LOCALISATION_RADII
)

# TODO: should we both for MESMER and for MESMER-X remove the
# residuals from the AR(1) process before calculating the covariance?
# is that we is happening in 'adjust_covariance_ar1'?
# TODO: using here weights from MESMER-X. I noticed that it affects the
# calculation in find_localized_empirical_covariance. Need to solve that.
localized_ecov = mesmer.stats.find_localized_empirical_covariance(
    data=stacked_targ[target_name],
    weights=stacked_weights.weight,
    localizer=phi_gc_localizer,
    dim="sample",
    k_folds=30,
)

localized_ecov["localized_covariance_adjusted"] = (
    mesmer.stats.adjust_covariance_ar1(
        localized_ecov.localized_covariance, local_ar_params.coeffs
    )
)

/Users/vbauer/Developer/mesmer/mesmer/core/utils.py:93: OptimizeWarning: No local minimum found, returning the last element
  warnings.warn("No local minimum found, returning the last element", OptimizeWarning)


In [28]:
file_end = f"{target_name}_{expr_name}_{esm}_{scenario}"
distrib_file = TEST_PATH / "distrib" / f"params_transform_distrib_{file_end}.nc"
local_ar_file = TEST_PATH / "local_variability" / f"params_local_AR_{file_end}.nc"
localized_ecov_file = (
    TEST_PATH / "local_variability" / f"params_localized_ecov_{file_end}.nc"
)

# if update_expected_files:
#     # save the parameters
#     transform_coeffs.to_netcdf(distrib_file)
#     local_ar_params.to_netcdf(local_ar_file)
#     localized_ecov.to_netcdf(localized_ecov_file)
#     pytest.skip("Updated param files.")

# else:
# load the parameters
expected_transform_params = xr.open_dataset(distrib_file)
xr.testing.assert_allclose(transform_coeffs, expected_transform_params)

expected_local_ar_params = xr.open_dataset(local_ar_file)
xr.testing.assert_allclose(
    local_ar_params["intercept"],
    expected_local_ar_params["intercept"],
    atol=1e-7,
)
xr.testing.assert_allclose(
    local_ar_params["coeffs"], expected_local_ar_params["coeffs"]
)
xr.testing.assert_allclose(
    local_ar_params["variance"], expected_local_ar_params["variance"]
)
xr.testing.assert_equal(
    local_ar_params["nobs"], expected_local_ar_params["nobs"]
)

expected_localized_ecov = xr.open_dataset(localized_ecov_file)
xr.testing.assert_allclose(localized_ecov, expected_localized_ecov)


AssertionError: Left and right Dataset objects are not close
Coordinates only on the right object:
    coefficient  <U2 8B ...

In [35]:
import numpy as np
np.testing.assert_allclose(transform_coeffs.c1.values, expected_transform_params.c1.values)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 21 / 118 (17.8%)
Max absolute difference among violations: 1.23141103e-05
Max relative difference among violations: 9.50247597e-06
 ACTUAL: array([ 8.990095e+00, -1.446269e+01,  8.113546e+01,  1.467111e+01,
       -7.141218e+00,  9.327718e+01, -2.188918e+01, -1.140058e+02,
        4.910474e+01, -4.541833e+01, -1.445842e+01,  4.921936e+01,...
 DESIRED: array([ 8.990094e+00, -1.446269e+01,  8.113546e+01,  1.467112e+01,
       -7.141209e+00,  9.327718e+01, -2.188918e+01, -1.140058e+02,
        4.910474e+01, -4.541834e+01, -1.445842e+01,  4.921936e+01,...